# Travaux pratiques 2

Vous avez découvert dans le TP précédent le jeu de données `ozone`, qui contient des valeurs manquantes. Dans ce TP, vous allez manipuler différentes méthodes d'imputation, pour prédire les valeurs manquantes et obtenir un jeu de données complet. Vous allez d'abord utiliser l'algorithme des $k$ plus proches voisins, puis des forêts aléatoires. Enfin, vous appliquerez une analyse en composantes principales, qui permet de traiter les valeurs manquantes et d'obtenir une meilleure représentation des données.

# Chargement des librairies

In [ ]:
library(VIM) # Pour la visualisation des données manquantes, et inclut une fonction d'imputation par plus-proches-voisins
library(randomForest) # Pour l'exemple simple sur les forêts aléatoires
library(missForest) # Fonction d'imputation par forêts aléatoires
library(missMDA) # Fonction d'imputation par ACP pour données continues et données mixtes
library(FactoMineR) # Représentation de l'ACP

# Chargement des données

Vous allez utiliser le jeu de données `ozone` du package `missMDA`, que vous avez déjà découvert dans le TP1. Pour rappel, ce jeu de données est composé de 112 mesures atmosphériques prises au niveau du sol à intervalles quotidiens, à Rennes (France) pendant l'été 2001, de début juin à fin septembre.

In [ ]:
data(ozone) # chargement des données
head(ozone) # affichage des 6 premières lignes

## Question 1

Comme dans le TP1, on utilise le package `VIM` pour visualiser les valeurs manquantes dans le jeu de données `ozone`. On affiche également les pourcentages de valeurs manquantes par variables. Y a-t-il des valeurs manquantes dans toutes les variables ? Quelle est la variable qui contient le plus de valeurs manquantes ?

In [ ]:
a <- aggr(ozone, plot = FALSE)
par(bg = "white") # définit le fond en blanc
plot(a, numbers = TRUE, prop = FALSE)

In [ ]:
a$missings["Percentage"] <- a$missings$Count / dim(ozone)[1]
a$missings

### Solution

Il y a des valeurs manquantes dans toutes les variables. Celle qui en contient le plus est la variable `T15`, la température à 15 heures, avec un pourcentage de valeurs manquantes à 17%.

## Question 2

Vous verrez dans la suite qu'il y a certaines méthodes d'imputation qui ne gèrent pas les données mixtes, lorsqu'il y a des variables continues et catégorielles. Dans le jeu de données `ozone`, y a-t-il des variables catégorielles ?

### Solution

Dans le jeu de données `ozone`, les 11 premières variables sont continues. Les deux dernières variables, nommées `vent` et `pluie`, sont catégorielles; elles indiquent respectivement l'orientation du vent et la présence de pluie.

Dans un premier temps, vous restreindrez votre analyse sur les variables continues. Dans un second temps (dernière partie du TP), vous appliquerez les méthodes d'imputation à tout le jeu de données (variables catégorielles inclues).

In [ ]:
ozone.mixt <- ozone
ozone <- ozone[, 1:11]

# Mesure de la qualité de la prédiction des valeurs manquantes

Dans la suite du TP, vous allez appliquer différentes méthodes d'imputation pour prédire les valeurs manquantes. Une des premières questions à se poser est de savoir comment quantifier la qualité de la prédiction.

L'erreur quadratique moyenne (EQM) permet de calculer la distance entre les valeurs prédites ($X^{\text{imp}}_{ij}$) et les valeurs ($X_{ij}$), comme suit:

$$\text{EQM}(X^{\text{imp}},X)=\frac{1}{n} \sum_{i=1}^n \sum_{j=1}^d (X^{\text{imp}}_{ij}-X_{ij})^2$$

Si on a deux tableaux, l'un imputé pour $X^{\text{imp}}$ et l'autre initial $X$, cette mesure compare les tableaux valeur par valeur (par exemple, $X^{\text{imp}}_{11}$ avec $X_{11}$ pour la première ligne/première colonne) et additionne finalement le tout.

En pratique, le problème est que vous n'avez pas accès aux valeurs ($X_{ij}$) qui se cachent derrière les `NA` de votre jeu de données.

## Question 3

Que proposez-vous ? Vous pouvez reprendre la dernière partie intitulée *Score de référence* du TP précédent.

### Solution

Une solution consiste à générer de nouvelles valeurs manquantes pour obtenir un jeu de données *amputé*. Ces nouvelles valeurs manquantes sont factices, et on peut calculer une erreur de prédiction en comparant la valeur prédite par la méthode d'imputation à la vraie valeur, qui est connue.

Dans la cellule de code suivante, nous créons un jeu de données ozone *amputé*, appelé `ozone.addNA`, de la même manière que dans le TP1, en introduisant de nouvelles valeurs manquantes selon les patterns les plus fréquents.

In [ ]:
set.seed(1)
ozone.full <- na.omit(ozone) # jeu de données avec seulement les lignes complètes
n.full.rows <- dim(ozone.full)[1]

# Patterns les plus fréquents
a <- aggr(ozone, plot = FALSE)
idx.frequent.pattern <- sort(a$percent, index.return = TRUE, decreasing = TRUE)
frequent.pattern <- matrix(0,nrow=5,ncol=dim(ozone)[2])
for (i in 1:5) {
  frequent.pattern[i,] <- a$tabcomb[idx.frequent.pattern$ix[i], ]
}
colnames(frequent.pattern)=colnames(ozone)
frequent.pattern <- frequent.pattern[-1,]
pattern.indices <- sample(1:nrow(frequent.pattern), size = n.full.rows, replace = TRUE) # on sélectionne les patterns

# Jeu de données ozone amputé
c <- 0
ozone.addNA <- ozone
for (i in 1:dim(ozone)[1]) {
  if (sum(is.na(ozone[i,]))==0){
    c <- c+1
    pattern <- frequent.pattern[pattern.indices[c], ]
    ozone.addNA[i, pattern == 1] <- NA
  }
}


Vous pouvez observez que le pourcentage de valeurs manquantes a bien augmenté pour certaines variables dans le jeu de données amputé.

In [ ]:
a <- aggr(ozone.addNA, plot = FALSE)
par(bg = "white") # définit le fond en blanc
plot(a, numbers = TRUE, prop = FALSE)

a$missings["Percentage"] <- a$missings$Count / dim(ozone.addNA)[1]
a$missings

La fonction de calcul de l'erreur quadratique moyenne est implémentée ci-dessous.

In [ ]:
mse <- function(xtrue, ximp){
  1/length(xtrue) * sum((ximp-xtrue)**2, na.rm = TRUE)
}

# Imputation par l'algorithme des $k$ plus proches voisins

## Question 4: un exemple simple

Prenons la première ligne du jeu de données `ozone.addNA`, qui correspond à la première mesure.

In [ ]:
ozone.addNA[1, ]

La seule variable manquante est la nébulosité à 15 heures (`Ne15`). L'idée va être de prédire cette valeur avec une valeur aggrégée sur les $k$ plus proches voisins de la mesure 1.

Ces plus proches voisins vont être choisis en calculant une distance entre deux mesures du jeu de données. On peut choisir la distance euclidienne, détaillée ci-dessous, qui permet de comparer deux mesures ($X_{i.}$ et $X_{\ell.}$), car les variables sont toutes continues.

$$d(X_{i.},X_{\ell.})=\sum_{j=1}^d (X_{ij}-X_{\ell j})^2$$

On calcule terme à terme la différence des valeurs entre la mesure $X_{i.}$ et la mesure $X_{\ell.}$.

Par exemple, si $X_{i.}=(1 \quad 2)$ et $X_{\ell.}=(1 \quad 0)$, le calcul détaillé est le suivant: $d(X_{i.},X_{\ell.})=(1-1)^2+(2-0)^2=0+2^2=4$.

### Question 4a

Dans le jeu de données `ozone`, peut-on simplement calculer la distance euclidienne de la première mesure avec toutes les autres ? Quelle peut-être une solution naïve ?

### Solution

Comme il y a des valeurs manquantes, on ne peut pas calculer la distance euclidienne. Une solution naïve est de calculer les distances euclidiennes en ne comptant que les variables qui sont observées pour les deux mesures.

Par exemple, si $X_{i.}=(1 \quad \text{NA} \quad 3)$ et $X_{\ell.}=(\text{NA} \quad 2 \quad 2)$, le calcul ne prend en compte que le dernier terme: $d(X_{i.},X_{\ell.})=(3-2)^2=1$.

C'est ce qui est implémenté dans la méthode d'imputation par les plus proches voisins.

On va également calculer ces distances euclidiennes parmis les mesures qui ont une nébulosité à 15 heures observée, car nous utiliserons cette information par la suite.

In [ ]:
ozone.Ne15.comp <- ozone.addNA[!is.na(ozone.addNA$Ne15), ]
dist.eucl <- c()
for (i in 2:dim(ozone.Ne15.comp)[1]){
  dist.eucl <- c(dist.eucl,sum((ozone.addNA[1, ] - ozone.Ne15.comp[i, ])**2, na.rm=TRUE))
}

### Question 4b

Les $k$-plus-proches voisins de la mesure 1 sont donc les mesures avec les plus petites distances euclidiennes. Prenons $k=5$, cela correspond aux mesures suivantes (mesures 13, 11, 2, 72 et 42).

In [ ]:
dist.eucl.ord <- order(dist.eucl) + 1
dist.eucl.ord[1:5]

Pour prédire la nébulosité à 15 heures sur la mesure 1, on va utiliser les nébulosité à 15 heures de ces mesures proches, en aggrégant leurs valeurs (par exemple, avec la médiane).

In [ ]:
knn.values <- c()
for (i in 1:5){
  knn.values <- c(knn.values, ozone.Ne15.comp[dist.eucl.ord[i], ]$Ne15)
}
median(knn.values)

La valeur prédite pour la nébulosité à 15 heures sur la mesure 1 sera donc 6.

Dans cette méthode d'imputation, quel va-être l'hyperparamètre, c'est-à-dire le paramètre à choisir par l'utilisateur ? Est-ce un choix qui peut avoir une influence sur les résultats ?

### Solution

L'hyperparamètre est le nombre $k$ de plus proches voisins. Il a une influence sur les résultats. Par exemple, si nous prenons $k=3$, la valeur prédite est 7.

In [ ]:
knn.values <- c()
for (i in 1:3) {
  knn.values <- c(knn.values, ozone.Ne15.comp[dist.eucl.ord[i], ]$Ne15)
}
median(knn.values)

Nous n'allons pas détailler cette méthode ici, mais une stratégie est de choisir $k$ en effectuant une validation croisée.

## Question 5: application de l'algorithme

Cette méthode d'imputation est implémentée dans la fonction `kNN` du package `VIM`. Le code suivant permet d'obtenir un jeu de données complet.

In [ ]:
ozone.addNA.kNN <- kNN(ozone.addNA, k = 15, imp_var = FALSE)

Le graphique ci-dessous représente le nuage de points des valeurs (observées et imputées) pour les variables `T12` et `T15`.

In [ ]:
mask <- is.na(ozone.addNA)

plot(ozone.addNA$T12, ozone.addNA$T15, col = "#d1e5f0", xlab = "T12", ylab = "T15")
points(ozone.addNA.kNN$T12[mask[, 3] == 1], ozone.addNA.kNN$T15[mask[, 3] == 1], col = "#2194ac")
points(ozone.addNA.kNN$T12[mask[, 4] == 1], ozone.addNA.kNN$T15[mask[, 4] == 1], col = "#2138ac")
points(ozone.addNA.kNN$T12[mask[, 3] == 1 & mask[, 4] == 1], ozone.addNA.kNN$T15[mask[, 3] == 1 & mask[, 4] == 1], col = "#ac6721")

legend("bottomright",
       legend = c("Complet", "X0 imputé", "X1 imputé", "X0,X1 imputés"),
       col = c("#d1e5f0", "#2194ac", "#2138ac", "#ac6721"),
       pch = 16,
       bty = "n"
)

Quelle méthode d’imputation simple peut servir de point de comparaison ?

### Solution

On peut comparer avec la méthode d'imputation par la moyenne que vous avez manipulée dans le TP1.

In [ ]:
impute.mean <- function(data) { #fonction d'imputation par la moyenne
  data.imputed <- data
  for (j in 1:11) {
    data.imputed[which(is.na(data[, j])), j] <- mean(data[, j], na.rm = TRUE)
  }
  return(data.imputed)
}

ozone.addNA.mean <- impute.mean(ozone.addNA)

In [ ]:
mask <- is.na(ozone.addNA)

plot(ozone.addNA$T12, ozone.addNA$T15, col = "#d1e5f0", xlab = "T12", ylab = "T15")
points(ozone.addNA.mean$T12[mask[, 3] == 1], ozone.addNA.mean$T15[mask[, 3] == 1], col = "#2194ac")
points(ozone.addNA.mean$T12[mask[, 4] == 1], ozone.addNA.mean$T15[mask[, 4] == 1], col = "#2138ac")
points(ozone.addNA.mean$T12[mask[, 3] == 1 & mask[, 4] == 1], ozone.addNA.mean$T15[mask[, 3] == 1 & mask[, 4] == 1], col = "#ac6721")

legend("bottomright",
  legend = c("Complet", "X0 imputé", "X1 imputé", "X0,X1 imputés"),
  col = c("#d1e5f0", "#2194ac", "#2138ac", "#ac6721"),
  pch = 16,
  bty = "n"
)

Graphiquement, on voit déjà que la méthode d'imputation par les plus proches voisins distord moins la distribution empirique des données. On peut aussi calculer les erreurs quadratiques moyennes.

In [ ]:
mse(ozone, ozone.addNA.kNN) # EQM de l'imputation par plus proches voisins

In [ ]:
mse(ozone, ozone.addNA.mean) # EQM de l'imputation par la moyenne

# Imputation itérative basée sur des forêts aléatoires

Dans cette partie, vous allez apprendre à utiliser une méthode d'imputation plus complexe, qui est itérative et basée sur des forêts aléatoires.

## Question 6: un exemple simple


En initialisation, l'imputation par la moyenne est appliquée au jeu de données. On obtient le jeu de données complet `ozone.init`.

In [ ]:
ozone.init <- impute.mean(ozone.addNA)

Ensuite, pour prédire les valeurs manquantes, l'algorithme *itère* sur les variables. Nous allons illustrer avec un exemple simple la méthode de prédiction basée sur des forêts aléatoires sur la première variable `maxO3`.

La première étape consiste à construire un sous-ensemble du jeu de données `ozone.init` avec les mesures pour lesquelles la variable `maxO3` est observée initialement; on appelle ce jeu de données `ozone.maxO3.comp`.

In [ ]:
ozone.maxO3.comp <- ozone.init[!is.na(ozone.addNA$maxO3), ]
head(ozone.maxO3.comp)

On crée également le sous-ensemble du jeu de données `ozone.init` avec les mesures pour lesquelles la variable `maxO3` est initialement manquante; on appelle ce jeu de données `ozone.maxO3.miss`.

In [ ]:
ozone.maxO3.miss <- ozone.init[is.na(ozone.addNA$maxO3), ]
head(ozone.maxO3.miss)

Notez que les valeurs de la variable `maxO3` sont toutes identiques. C'est tout à fait normal car une imputation par la moyenne a été appliquée pour obtenir `ozone.init` à partir duquel on construit le sous-ensemble du jeu de données.

### Question 6a

Sur quel jeu de données construire une forêt aléatoire pour prédire les valeurs manquantes de la variable `maxO3` à partir des autres ?

### Solution

On construit un modèle avec une forêt aléatoire sur le jeu de données `ozone.maxO3.comp` pour prédire la variable `maxO3` en fonction des autres, c'est-à-dire on apprend la fonction $f$ telle que: $\text{maxO3}=f(\text{T9, T12, T15, Ne9, Ne12, Ne15, Vx9, Vx12, Vx15, maxO3v})$. Pour utiliser une forêt aléatoire avec 500 arbes (paramètre `ntree`), l'implémentation est la suivante.

In [ ]:
modele_rf <- randomForest(maxO3 ~ ., data = ozone.maxO3.comp, ntree = 500)

On applique ensuite ce modèle sur le jeu de données `ozone.maxO3.miss` pour prédire les valeurs manquantes de `maxO3`.

In [ ]:
predict(modele_rf, newdata = ozone.maxO3.miss[, 2:11])

Le jeu de données avec les valeurs prédites pour la variable `maxO3` est appelé `ozone.maxO3.imp1`.

In [ ]:
ozone.maxO3.imp1 <- ozone.init
ozone.maxO3.imp1[is.na(ozone.addNA$maxO3), ]$maxO3 <- predict(modele_rf, newdata = ozone.init[is.na(ozone.addNA$maxO3), 2:11])

### Question 6b

L'algorithme passe ensuite à la seconde variable `T9`. Le but est alors de prédire les valeurs manquantes de `T9` en fonction des autres variables. De même que précédemment, on va construire deux sous-ensembles du jeu de données, l'un avec les mesures pour lesquelles `T9` est initialement observée, l'autre avec les mesures pour lesquelles `T9` est initialement manquante. À partir de quel jeu de données ces deux sous-ensembles sont construits ?

### Solution

C'est à partir du jeu de données `ozone.maxO3.imp1` que les deux sous-ensembles sont construits. Pour cette seconde étape, on prend en compte les prédictions déjà effectuées sur la variable `maxO3`.

L'algorithme itère ensuite sur toutes les variables jusqu'à convergence; c'est en cela que cette méthode est itérative.

## Question 7: application de l'algorithme

Cette méthode d'imputation est implémentée dans la fonction `missForest` du package `missForest`. Le code suivant permet d'obtenir un jeu de données complet.

In [ ]:
ozone.addNA.forest <- missForest(ozone.addNA, maxiter = 10, ntree = 100)$ximp

Le graphique ci-dessous représente le nuage de points des valeurs (observées et imputées) pour les variables `T12` et `T15`.

In [ ]:
mask <- is.na(ozone.addNA)

plot(ozone.addNA$T12, ozone.addNA$T15, col = "#d1e5f0", xlab = "T12", ylab = "T15")
points(ozone.addNA.forest$T12[mask[, 3] == 1], ozone.addNA.forest$T15[mask[, 3] == 1], col = "#2194ac")
points(ozone.addNA.forest$T12[mask[, 4] == 1], ozone.addNA.forest$T15[mask[, 4] == 1], col = "#2138ac")
points(ozone.addNA.forest$T12[mask[, 3] == 1 & mask[, 4] == 1], ozone.addNA.forest$T15[mask[, 3] == 1 & mask[, 4] == 1], col = "#ac6721")

legend("bottomright",
       legend = c("Complet", "X0 imputé", "X1 imputé", "X0,X1 imputés"),
       col = c("#d1e5f0", "#2194ac", "#2138ac", "#ac6721"),
       pch = 16,
       bty = "n"
)

L'erreur quadratique moyenne est calculée ci-dessous.

In [ ]:
mse(ozone, ozone.addNA.forest) # EQM de l'imputation par forêts aléatoires

Comparez avec les résultats obtenus par l'imputation avec l'algorithme des $k$-plus-proches-voisins.

### Solution

L'erreur quadratique moyenne est largement plus faible avec cette méthode d'imputation. Cela peut s'expliquer par le fait qu'elle tient mieux compte des relations entre les variables, en construisant un modèle de prédiction par variable, en fonction des autres. L'utilisation des forêts aléatoires est aussi un outil puissant pour construire des modèles flexibles et complexes (par exemple, la relation entre les variables n'est pas restreinte à une relation linéaire). L'imputation avec l'algorithme des $k$-plus-proches-voisins n'utilise que des aggrégations des valeurs des données.

# Analyse en composantes principales en présence de `NA`

Une autre méthode d'imputation est basée sur l'Analyse en Composantes Principales (ACP). Elle permet de (i) représenter les données dans un espace de dimension réduite, et (ii) de prédire les valeurs manquantes à partir de cette nouvelle représentation.

On peut utiliser le package `missMDA`.

## Question 8

Selon vous, quel est l'hyperparamètre principal de l'algorithme pour obtenir un espace de dimension réduite ?

### Solution

Dans cette méthode d'imputation, l'hyperparamètre principal est la dimension de l'espace réduit dans lequel sont représentées les données, c'est-à-dire le nombre de composantes principales de l'ACP. Il existe une fonction `estim_ncpPCA` qui permet d'effectuer une validation croisée pour estimer ce paramètre.

In [ ]:
nb.comp.quant <- estim_ncpPCA(ozone.addNA, ncp.max = 11)
nb.comp.quant$ncp

La fonction `imputePCA` permet d'obtenir un jeu de données complet. On calcule ensuite l'erreur quadratique moyenne, qui est comparable à celle obtenue avec la méthode d'imputation basée sur des forêts aléatoires.

In [ ]:
res.comp.quant <- imputePCA(ozone.addNA, ncp = nb.comp.quant$ncp)
ozone.addNA.PCA <- res.comp.quant$completeObs
head(ozone.addNA.PCA)

In [ ]:
mse(ozone, ozone.addNA.PCA)

## Question 9

Vous pouvez aussi obtenir une représentation des données dans l'espace de dimension réduite estimé par l'ACP.

Le graphique suivant est le cercle des corrélations des variables avec les deux premières composantes
principales. On peut y lire comment chaque variable intervient dans les différentes composantes estimées par l'ACP:

* Plus une variable est proche d’un axe, plus elle a été utilisée pour construire cette composante principale de l'ACP.

* Plus la flèche est longue, plus elle a eu un poids important.

On peut aussi y lire les corrélations entre les variables:

* Plus deux variables sont corrélées, plus leurs flèches pointent dans la même direction (l'angle entre les flèches de ces deux variables est petit).

In [ ]:
 res.pca <- PCA(res.comp.quant$completeObs)

Interprétez le graphique.

### Solution

La première composante principale de l'ACP semble très reliée aux mesures de nébulosité, tandis que la deuxième composante principale semble construite en utilisant un mixte de toutes les autres variables.

Les variables de nébulosité sont très corrélées entre elles, de même que les variables de vent, ainsi que les variables de températures avec le maximum d'ozone.

# Méthodes d'imputation sur données mixtes

Les trois méthodes d'imputation vues précédemment sont directement applicables sur un jeu de données mixte, qui comprend à la fois des variables continues et catégorielles. Dans cette dernière partie, vous allez appliquer des méthodes d'imputation au jeu de données `ozone` entier, qui comprend deux variables catégorielles (`vent` et `pluie`).

In [ ]:
ozone.addNA.mixt <- ozone.addNA
ozone.addNA.mixt['vent'] <- ozone.mixt$vent
ozone.addNA.mixt['pluie'] <- ozone.mixt$pluie
head(ozone.addNA.mixt)

## Question 10

Pour la méthode d'imputation par l'algorithme des $k$ plus proches voisins, quelle est la difficulté dans le cas d'un jeu de données mixte ?

### Solution

L'algorithme des $k$ plus proches voisins calcule une distance entre des points. La distance euclidienne que vous avez utilisé en question 4 ne fait plus sens s'il y a des variables catégorielles. Dans la fonction `kNN` du package `VIM`, la distance de Gower est utilisée ([lien wikipédia](https://en.wikipedia.org/wiki/Gower%27s_distance) pour plus de détails sur cette distance). Elle permet de mesurer la distance de deux objets mixtes.

Pour la méthode d'imputation basée sur des forêts aléatoires, l'avantage est qu'elle est très flexible, puisque un modèle est appris par variable. On peut directement utiliser la fonction `missForest`.

Enfin, pour l'analyse en composantes principales, il existe la fonction `imputeFAMD` pour les jeux de données mixtes du package `missMDA`.

Pour obtenir un jeu de données complet, voici les implémentations des différentes méthodes, lorsque toutes les variables sont considérées. Les erreurs quadratiques moyennes ne sont calculées que pour les variables continues.

In [ ]:
# imputation par plus proches voisins
ozone.addNA.mixt.kNN <- kNN(ozone.addNA.mixt, k = 15, imp_var = FALSE)

# erreur quadratique moyenne
mse(ozone.mixt[, 1:11], ozone.addNA.mixt.kNN[, 1:11])

In [ ]:
# imputation par forêts aléatoires
ozone.addNA.mixt.forest <- missForest(ozone.addNA.mixt, maxiter = 10, ntree = 100)$ximp

# erreur quadratique moyenne
mse(ozone.mixt[, 1:11], ozone.addNA.mixt.forest[, 1:11])

In [ ]:
# imputation par analyse en composantes principales (mixte)
nb.comp.mixt <- estim_ncpFAMD(ozone.addNA.mixt, ncp.max = 13)
res.comp.mixt <- imputeFAMD(ozone.addNA.mixt, ncp = nb.comp.mixt$ncp)
ozone.addNA.mixt.FAMD <- res.comp.mixt$completeObs

# erreur quadratique moyenne
mse(ozone[, 1:11], ozone.addNA.mixt.FAMD[, 1:11])

# représentation graphique
res.afdm <- FAMD(ozone.addNA.mixt, tab.disj = res.comp.mixt$tab.disj)